# Data Analytics YTIP2200, part3 - exercises

Return .ipynb file to Optima Return box Sun 21th Feb at latest.
Write in comment field which excercised you've done fully/partly. You can also comment which parts are done/missing.

You can write your solutions in new code cells between existing cells.

Don't use for loops etc.


# 1a. Municipalities
Read to dataframe a text file containing data from Finnish municipalities (2017-19). Field separator is semicolon and decimal separator comma.

http://gpspekka.kapsi.fi/dataanalytics/part3/muncipalities.txt

a) Calculate *total area* for each *sub-region* and print 5 sub-regions with largest total area (sub-region name and total area)

b) Calculate for each *region* percentage of region's population living in *towns* (as *municipality type*).   
Print 5 largest percentages (region name, percentage, population in towns, total population).

c) Calculate for each *sub-region* percentage of Swedish speaking people.  
Note: can't use average of "Share of Swedish speakers%" since municipality populations vary).  
Print 5 largest values (sub-region name and precentage of Swedish speaking people).
 
 
# 1b. access.log
Read 3-hour access.log from one server to DataFrame (15 MB zip, 600 000 rows).  

http://gpspekka.kapsi.fi/accesslog2.zip

Part 3 theory has an axample on access.log  reading.

Fields

    ip address
    ident-id (-)
    userid (-)
    time
    request
    status
    size
    referer
    user agent

From time field parse hours and minutes and for each minute calculate how many **uniuqe ip addresses** there were.
(No need to parse that on timestamps, strings work fine here.)

List 10 minutes with most unique ip addresses.

Expected result:
```
12:55    792  
12:51    789  
12:54    789  
12:53    787  
12:56    786  
12:52    783
14:00    773
14:01    769
13:59    762
12:50    760
```

In [2]:
import pandas as pd
import numpy as np

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

print('1aa) \n')

df = pd.read_csv('http://gpspekka.kapsi.fi/dataanalytics/part3/muncipalities.txt', sep=';', decimal=',')

# Five sub-regions with largest Total area
print(df.groupby('Sub-region')['Total area'].sum().nlargest(5))

1aa) 

Sub-region
Pohjois-Lapin seutukunta    35121.02
Itä-Lapin seutukunta        21726.54
Tunturi-Lapin seutukunta    21312.16
Kehys-Kainuun seutukunta    15434.48
Oulunkaaren seutukunta      12240.43
Name: Total area, dtype: float64


In [3]:
print('\n 1ab) \n')

# Calculate for each region percentage of region's population living in towns (as municipality type).
# Print 5 largest percentages (region name, percentage, population in towns, total population).

# Ryhmitellään ja summataan populaatio per ryhmä
regions = df.groupby(['Region', 'Municipality type']).agg({'Population': 'sum'})

# Uudet sarakkeet 
regions['Percentage'] = regions.groupby(level=0)['Population'].apply(lambda x: 100 * x / x.sum())
regions['Total population'] = regions.groupby(level=0)['Population'].apply(lambda x: x/x * x.sum())

# reset index, jotta pääsee käsiksi kaikkiin sarakkeisiin
flat = regions.reset_index()

# Viisi isointa Townia
clean = flat[flat['Municipality type']=='Town'].nlargest(5, 'Percentage')

# reorder and "drop" extra
clean = clean[['Region', 'Percentage', 'Population', 'Total population']]

# rename columns
clean.columns = ['Region', 'Percentage', 'Population in towns', 'Total population']

print(clean)


# Notes - talteen itselle matkan varrelta
# x is group here instead of Series and adding the second new column with same idea will fail (group vs. series https://stackoverflow.com/questions/48479558/clarification-on-lambda-operator-in-pandas-groupby)
# population_by_region_mun['Percentage'] = population_by_region_mun.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
# population_by_region_mun['Total'] = population_by_region_mun.groupby(level=0).apply(lambda x: x/x * float(x.sum()))
# print(regions.reset_index().nlargest(5, 'Percentage'))


 1ab) 

             Region  Percentage  Population in towns  Total population
17      Kymenlaakso   90.171177               156346          173388.0
35          Uusimaa   86.882115              1451821         1671024.0
21        Pirkanmaa   84.496840               435239          515095.0
37  Varsinais-Suomi   82.700352               395789          478582.0
5   Etelä-Pohjanmaa   81.707298               155011          189715.0


In [4]:
print('\n 1ac) \n')

# Calculate for each sub-region percentage of Swedish speaking people.
# Note: can't use average of "Share of Swedish speakers%" since municipality populations vary).
# Print 5 largest values (sub-region name and precentage of Swedish speaking people).


df['Swedish speakers'] = df['Population'] * df['Share of Swedish speakers%'] / 100.0

subr = df.groupby('Sub-region').agg({'Population': 'sum', 
                             'Swedish speakers': 'sum'})

subr['Percentage of swedish speakers'] = subr['Swedish speakers'] / subr['Population']
subr = subr.drop(['Swedish speakers', 'Population'], axis = 1)
print(subr.nlargest(5, 'Percentage of swedish speakers'))


 1ac) 

                                    Percentage of swedish speakers
Sub-region                                                        
Ålands landsbygd                                          0.896688
Mariehamns stads ekonomiska region                        0.835000
Ålands skärgård                                           0.829686
Jakobstadsregionen                                        0.747392
Sydösterbottenin seutukunta                               0.663923


In [5]:
print('1b) \n')
# From time field parse hours and minutes and for each minute calculate how many uniuqe ip addresses there were. 
# (No need to parse that on timestamps, strings work fine here.)
# List 10 minutes with most unique ip addresses.

df_log = pd.read_csv('http://gpspekka.kapsi.fi/accesslog2.zip', 
                sep = ' ', 
                quotechar = '"',
                usecols = [0,3],  # there are no column labels yet, so need to give implicit integer (column) indices
                names = ['ip', 'date'],
                converters = {'date' : (lambda x : x[13:18])} # otetaan date-stringistä 13. ja 18. merkin väli
                )

# df_log.head()

df_log.groupby('date')['ip'].nunique().nlargest(10)

1b) 



date
12:55    792
12:51    789
12:54    789
12:53    787
12:56    786
12:52    783
14:00    773
14:01    769
13:59    762
12:50    760
Name: ip, dtype: int64

# 2. Crosstab

File http://gpspekka.kapsi.fi/dataanalytics/part3/query.csv includes results of imaginary query to company staff.

Education-field has values:

    1 = basic education 
    2 = upper secondary education
    3 = higher education
    4 = third-cycle degree

Management and Tasks fields decribe satisfaction to management and work tasks in scale 1-5.

Health, Fitness and Timeshare fields have one if person has used these services, otherwise NaN.

Aggregate/crosstab/pivot: 

a) counts with *Family situation* in rows, and *satisfaction to management* in columns (i.e. how many singles have answered on *Management* 1, 2, 3 etc.)

b) percentage shares of *satisfaction to management* with *education as text* in rows and share of 1, 2, 3 ... in columns (i.e. how many % of "basic education" -rows have answered 1, 2, 3, 4 and 5 etc).

c) averages of *satisfaction to management*  and *satisfaction to work tasks* with *age group* (20-29, 30-39, ... , 60-69) in rows

d) percentage of men and women on usage of *Health*, *Fitness* and *Timeshare*
(what share of men have used health services etc)

Model (labels may be different):


![model](http://gpspekka.kapsi.fi/dataanalytics/part3/model2.png)

In [6]:
print('2A) \n')
data2 = pd.read_csv('http://gpspekka.kapsi.fi/dataanalytics/part3/query.csv', sep=',', decimal=',')
print(data2)
print(pd.crosstab(data2['Family situation'], data2['Management']))

print('2B) \n')
data2b = pd.crosstab(data2['Management'], data2['Education']).apply(lambda x: x / float(x.sum())).T

pd.options.display.float_format = '{:.1%}'.format
print(data2b)

print('2C) \n')

pd.options.display.float_format = '{:.6f}'.format
data2c = data2


# Lisätään agegroups -sarake
data2c['AgeGroups']= pd.cut(data2c['Age'], np.arange(20, 71, 10))
c_ans = data2c.groupby(['AgeGroups']).agg({'Management': 'mean','Tasks': 'mean'})
print(c_ans)


print('2D) \n')

pd.options.display.float_format = '{:.1%}'.format

data2d = data2.groupby(['Sex']).agg({'Health': 'count','Fitness': 'count','Timeshare': 'count'})

# Seuraaville 4 riville voisi olla tyylikkäämpi ratkaisu...
females = len(data2[data2['Sex'] == 'female'])
males = len(data2[data2['Sex'] == 'male'])
data2d.iloc[0,:] = data2d.iloc[0,:] / females
data2d.iloc[1,:] = data2d.iloc[1,:] / males

print(data2d)

2A) 

    Number     Sex  Age Family situation  Education  palveluv  Management  Tasks  Health  Fitness  Timeshare
0        1  female   38           single        1.0      22.0           3      3     NaN      NaN        NaN
1        2  female   29      with family        2.0      10.0           1      3     NaN      NaN        NaN
2        3    male   30           single        1.0       7.0           3      3     1.0      NaN        NaN
3        4    male   36      with family        1.0      14.0           3      3     1.0      NaN        NaN
4        5  female   24           single        2.0       4.0           2      2     1.0      NaN        NaN
..     ...     ...  ...              ...        ...       ...         ...    ...     ...      ...        ...
77      78    male   22           single        3.0       0.0           4      4     NaN      1.0        1.0
78      79    male   33           single        1.0       2.0           1      2     1.0      NaN        NaN
79      80   

# 3. Breaks
File  http://gpspekka.kapsi.fi/dataanalytics/part3/machine.csv (11 MB) contains data from a machine with 10 second interval. There is timestamp and field *break* that has value 1 when mahcine has been out of use and 0 when device has been running.

Pick data on breaks i.e. consecutive 1-rows. Make DataFrame that has for each break:

    break number
    first timestamp of the break
    last timestamp of the break
    duration of the break in hours (line count divided by 360)
    
10 first breaks should look like: 


```
num start 	                        end 	                        duration (h)			
1 	2019-03-18T11:26:30.000000Z 	2019-03-18T11:34:30.000000Z 	0.136111
2 	2019-03-18T11:35:30.000000Z 	2019-03-18T11:43:00.000000Z 	0.127778
3 	2019-03-18T14:05:30.000000Z 	2019-03-18T14:16:50.000000Z 	0.191667
4 	2019-03-18T21:49:00.000000Z 	2019-03-18T21:56:00.000000Z 	0.119444
5 	2019-03-18T22:51:20.000000Z 	2019-03-18T22:56:20.000000Z 	0.086111
6 	2019-03-18T22:57:20.000000Z 	2019-03-18T22:59:40.000000Z 	0.041667
7 	2019-03-19T01:47:40.000000Z 	2019-03-19T02:05:30.000000Z 	0.300000
8 	2019-03-19T02:54:40.000000Z 	2019-03-19T02:58:40.000000Z 	0.069444
9 	2019-03-19T03:54:00.000000Z 	2019-03-19T04:00:00.000000Z 	0.102778
10	2019-03-19T04:09:10.000000Z 	2019-03-19T04:20:40.000000Z 	0.194444

```

In [7]:
print('3) \n')

df_breaks = pd.read_csv('http://gpspekka.kapsi.fi/dataanalytics/part3/machine.csv', 
                sep = ','
#                parse_dates = [0]
                )



3) 



# 4. Snow winters

File http://gpspekka.kapsi.fi/dataanalytics/part2/weather.txt has weather data from Jyväskylä 1959-2021. Read file to DataFrame and calculate "snow sum" for each winter by adding each day's snow depth ("snow sum" is not any meteorological term).

Start from winter 1959-60 and end to 2019-20 since 1958-59 and 2020-21 are only partial.

Notes:
* it is by winter, not by year. Make for example 1st of August as limit.
* FMI uses -1 as snow depth when "there is absolutely no snow at all". We don't want to reduce snow sum with that so replace -1 -> 0
* for some reason there are few missing snow depths on the data. If there has been snow that gives wrong sum. It is safely to assume that snow depth has been approximately same as previous day so fill NaNs with previous valid value

Then produce DataFrame that has winter as index (in form "1959-1960") and columns
* snow sum
* snow sum rank among winters so that largest = 1
* count of days that snow depth has been over zero
* largest snow depth of the winter


10 first and 10 last should look like:
```
           Snow sum  rank  count  max
Winter                               
1959-1960      5593    18    169   65
1960-1961      5082    28    162   60
1961-1962      6644    12    156   78
1962-1963      3984    43    158   50
1963-1964      4512    38    166   49
1964-1965      5465    20    159   62
1965-1966      9053     3    185   86
1966-1967      5818    16    164   74
1967-1968      5150    25    161   64
1968-1969      7863     5    195   90
           Snow sum  rank  count  max
Winter                               
2010-2011      6670    11    174   71
2011-2012      4582    37    148   59
2012-2013      5231    22    153   59
2013-2014       644    61     95   17
2014-2015      3314    46    134   47
2015-2016      2002    55    110   40
2016-2017      2500    54    153   38
2017-2018      6882     8    161   81
2018-2019      4030    42    150   54
2019-2020      1432    59    112   30
```

In [8]:
print('4) \n')

df_weather = pd.read_csv('http://gpspekka.kapsi.fi/dataanalytics/part2/weather.txt', 
                sep = ',',
                decimal='.',
                usecols = [0, 1, 2, 6],  
                # parse_dates = [['Year', 'Month', 'Day']]
                )
# FMI uses -1 as snow depth when "there is absolutely no snow at all". 
# We don't want to reduce snow sum with that so replace -1 -> 0
df_weather['Snow depth (cm)'].replace(-1, 0, inplace = True)


# Start from winter 1959-60 and end to 2019-20, since 1958-59 and 2020-21 are only partial.
# Clean 
is_year = df_weather['Year'] == 1959
is_month = df_weather['Month'] < 8
df_weather = df_weather[~(is_year & is_month)]

# Clean
is_year = df_weather['Year'] == 2020
is_month = df_weather['Month'] > 7
df_weather = df_weather[~(is_year & is_month)]
df_weather = df_weather[df_weather['Year'] != 2021]

# Replace few missing snow depths in the data by filling NaNs with previous valid value
df_weather = df_weather.fillna(method='ffill')

# Labels for seasons
start_years = list(range(1959,2020))
seasons = [str(x) + '-' + str(x+1) for x in start_years]

# Bins. Rangen alalaita eka index, korotus 366, paitsi karkausvuonna yksi enemmän 
# Rangen ylälaita kaivettu kokeilemalla ja laittamalla yksi yli viimeisen bin:n...)
bins = np.arange(210, 22537, 366)
for i in range(0, len(bins)):
    if i % 4 == 0:
        bins[i] += 1

# print(bins)

# Lisätään sarake talvikausille
df_weather['Winters']= pd.cut(df_weather.index, bins, labels = seasons)
#print(df_weather)

# Jos lunta, laitetaan päivälle arvoksi 1, muuten 0
df_weather['Winter days']= np.where(df_weather['Snow depth (cm)'] > 0, 1, 0)


df_agg = df_weather.groupby('Winters').agg({'Snow depth (cm)': ['sum','max'],'Winter days': 'sum'}).reset_index()
df_agg['rank'] = df_agg['Snow depth (cm)']['sum'].rank(ascending=False).astype(int)


# set the columns to the top level, to flatten hierarchical index in columns
df_agg.columns = df_agg.columns.get_level_values(0)

# rename columns
df_agg.columns = ['Winter', 'Snow sum', 'max', 'count', 'rank']

# cleaning floats to int
df_agg['Snow sum'] = df_agg['Snow sum'].astype(int)
df_agg['max'] = df_agg['max'].astype(int)

# setting Winter as index
df_agg = df_agg.set_index('Winter')

# reorder columns
df_agg = df_agg[['Snow sum', 'rank', 'count', 'max']]

print(df_agg.head(10))
print(df_agg.tail(10))



# Itselle talteen

# Start from winter 1959-60 and end to 2019-20, since 1958-59 and 2020-21 are only partial.
# df_weather = df_weather[(df_weather['Year_Month_Day'] > '1959-07-31') & (df_weather['Year_Month_Day'] < '2020-08-01')]

# korvaaminen onnistuu näinkin, mutta hoidettu yläpuolella matskussa esitellyllä replacellä
# df_weather.loc[df_weather['Snow depth (cm)'] == -1, 'Snow depth (cm)'] = 0 
# df_weather.tail(40)

# NaNien tarkistamiseen - onko niitä, missä? onnistuiko täyttö?
# print(df_weather[df_weather.isna().any(axis=1)])
# näin voi printata tietyssä indeksissä olevan rivin 
# print(df_weather.loc[[17350]])

# Summed by month and reset index
# df_monthly_sums = df_weather.groupby(['Year', 'Month']).agg('Snow depth (cm)').sum().reset_index()

# Count months for range in binning
# months = df_monthly_sums['Month'].count()

# Add seasons and bin them by month
# df_monthly_sums['Winters'] = pd.cut(df_monthly_sums.index, np.arange(-1, months, 12), labels = seasons)

# Hierarkisen indexin vuoksi ajatus kopioida ja poistaa, mutta meni rumaksi, hankalaksi ja olisi pitänyt toistaa
# df_agg['count'] = df_agg['Winter days']['sum']
# df_agg.drop(['Winter days'], axis = 1, inplace=True)

4) 

           Snow sum  rank  count  max
Winter                               
1959-1960      5593    18    169   65
1960-1961      5082    28    162   60
1961-1962      6644    12    156   78
1962-1963      3984    43    158   50
1963-1964      4512    38    166   49
1964-1965      5465    20    159   62
1965-1966      9053     3    185   86
1966-1967      5818    16    164   74
1967-1968      5150    25    161   64
1968-1969      7863     5    195   90
           Snow sum  rank  count  max
Winter                               
2010-2011      6670    11    174   71
2011-2012      4582    37    148   59
2012-2013      5231    22    153   59
2013-2014       644    61     95   17
2014-2015      3314    46    134   47
2015-2016      2002    55    110   40
2016-2017      2500    54    153   38
2017-2018      6882     8    161   81
2018-2019      4030    42    150   54
2019-2020      1432    59    112   30


,Year,Month,Day,Snow depth (cm),Winters,Winter days
212,1959,8,1,0.0%,1959-1960,0
213,1959,8,2,0.0%,1959-1960,0
214,1959,8,3,0.0%,1959-1960,0
215,1959,8,4,0.0%,1959-1960,0
216,1959,8,5,0.0%,1959-1960,0
...,...,...,...,...,...,...
22488,2020,7,27,0.0%,2019-2020,0
22489,2020,7,28,0.0%,2019-2020,0
22490,2020,7,29,0.0%,2019-2020,0
22491,2020,7,30,0.0%,2019-2020,0


# 5. League Table 
File http://gpspekka.kapsi.fi/dataanalytics/part3/epl1718.txt has English Premier league results from season 2017-18.

Read it to DataFrame and generate league table, which has team name as index and columns:
* games played
* wins
* draws
* defeats
* goals for - goals against
* points

(win gives 3 points and draw gives 1 point)

Sort it with points (most points win). If points are equal, then sorted by
* goal difference (goals for - goals against)
* goals for

Fields needed:
 * HomeTeam
 * AwayTeam
 * FTHG (fulltime home goals)
 * FTAG (fulltime away goals)

Expected result:
```
                games  wins  draws  defeats   goals  points
Man City           38    32      4        2  106-27     100
Man United         38    25      6        7   68-28      81
Tottenham          38    23      8        7   74-36      77
Liverpool          38    21     12        5   84-38      75
Chelsea            38    21      7       10   62-38      70
Arsenal            38    19      6       13   74-51      63
Burnley            38    14     12       12   36-39      54
Everton            38    13     10       15   44-58      49
Leicester          38    12     11       15   56-60      47
Newcastle          38    12      8       18   39-47      44
Crystal Palace     38    11     11       16   45-55      44
Bournemouth        38    11     11       16   45-61      44
West Ham           38    10     12       16   48-68      42
Watford            38    11      8       19   44-64      41
Brighton           38     9     13       16   34-54      40
Huddersfield       38     9     10       19   28-58      37
Southampton        38     7     15       16   37-56      36
Swansea            38     8      9       21   28-56      33
Stoke              38     7     12       19   35-68      33
West Brom          38     6     13       19   31-56      31
```

In [8]:
# run into solution linked below with functions and cool grouping, while researching for this task and it was so nicely done, 
# I could not go with any other solution
# edited from source https://alysivji.github.io/pl-analysis-part1-finding-longest-consecutive-position-streak.html


df_league = pd.read_csv('http://gpspekka.kapsi.fi/dataanalytics/part3/epl1718.txt', 
                sep = ',',
                usecols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'Referee'],
                )


df_league.head()

df_league['H'] = df_league['HomeTeam']
df_league['A'] = df_league['AwayTeam']

cols_to_keep = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG',
                'FTAG', 'Referee']

team_results = pd.melt(
    df_league, 
    id_vars=cols_to_keep, 
    value_vars=['H', 'A'],
    var_name='Home/Away',
    value_name='Team')


team_results['Opponent'] = np.where(team_results['Team'] == team_results['HomeTeam'],
                                    team_results['AwayTeam'],
                                    team_results['HomeTeam'])

points_map = {
    'W': 3,
    'D': 1,
    'L': 0
}

def get_result(score, score_opp):
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'


# full time goals
team_results['Goals'] = np.where(team_results['Team'] == team_results['HomeTeam'],
                                 team_results['FTHG'],
                                 team_results['FTAG'])
team_results['Goals_Opp'] = np.where(team_results['Team'] != team_results['HomeTeam'],
                                     team_results['FTHG'],
                                     team_results['FTAG'])
team_results['Result'] = np.vectorize(get_result)(team_results['Goals'], team_results['Goals_Opp'])
team_results['points'] = team_results['Result'].map(points_map)

  
# Drop unnecessary columns and sort by date
cols_to_drop = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']
team_results = (team_results
                    .drop(cols_to_drop, axis=1)
                    .sort_values(by=['Date', 'Referee']))
    
# Testing the standings function 
(team_results
     .groupby('Team')
     .sum()['points']
     .sort_values(ascending=False))

def standings(frame, result_col, goals_col, goals_opp_col, points_col):
    """This function takes in a DataFrame and strings identifying fields
    to calculate the league table.
    
    Making it generalized will allow us to calculate league tables for
    First Half Goals only. Second Half Goals only.
    """
    record = {}
    
    record['games'] = np.size(frame[result_col])
    record['wins'] = np.sum(frame[result_col] == 'W')
    record['draws'] = np.sum(frame[result_col] == 'D')
    record['defeats'] = np.sum(frame[result_col] == 'L')
    record['goals'] = str(np.sum(frame[goals_col])) + '-' + str(np.sum(frame[goals_opp_col]))
    record['points'] = np.sum(frame[points_col])
    
    
    return pd.Series(record,
                     index=['games', 'wins', 'draws', 'defeats', 'goals', "points"])

# Get League Table
results_byteam = team_results.groupby(['Team'])

(results_byteam 
     .apply(standings,
            result_col='Result',
            goals_col='Goals',
            goals_opp_col='Goals_Opp',
            points_col='points')
     .sort_values('points', ascending=False))





,games,wins,draws,defeats,goals,points
Team,,,,,,
Man City,38,32,4,2,106-27,100
Man United,38,25,6,7,68-28,81
Tottenham,38,23,8,7,74-36,77
Liverpool,38,21,12,5,84-38,75
Chelsea,38,21,7,10,62-38,70
Arsenal,38,19,6,13,74-51,63
Burnley,38,14,12,12,36-39,54
Everton,38,13,10,15,44-58,49
Leicester,38,12,11,15,56-60,47
